<p><img alt="Colaboratory logo" height="45px" src="/img/colab_favicon.ico" align="left" hspace="10px" vspace="0px"></p>

#<h1>Import packages</h1>

In [1]:
import easydict
import os
import numpy as np
import math
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch
import torchvision

import pandas as pd
from tqdm import tqdm 
import time

from random import randint
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import animation
from IPython.display import HTML
from matplotlib import rc
%matplotlib inline

import copy as cp
from IPython.display import clear_output

 
use_cuda = True
if torch.cuda.is_available():
  device = torch.device("cuda")


np.set_printoptions(edgeitems=30, linewidth=100000, formatter=dict(float=lambda x: "%.3g" % x))

cuda = True if torch.cuda.is_available() else False
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

<p><img alt="Colaboratory logo" height="45px" src="/img/colab_favicon.ico" align="left" hspace="10px" vspace="0px"></p>

#<h1>Terrain data organization</h1>

In [2]:
# Load terrain T.O.W.R data
dataFlat = pd.read_csv('../src/data/csv/NewDataset/02.Terrain_data_flat.txt')
dataFlat

,PathNum,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,300
0,1,-5,-5.0,0,-4.9,-5.0,0,-4.8,-5.0,0,...,0,4.7,-5.0,0,4.8,-5.0,0,4.9,-5.0,0
1,1,-5,-4.9,0,-4.9,-4.9,0,-4.8,-4.9,0,...,0,4.7,-4.9,0,4.8,-4.9,0,4.9,-4.9,0
2,1,-5,-4.8,0,-4.9,-4.8,0,-4.8,-4.8,0,...,0,4.7,-4.8,0,4.8,-4.8,0,4.9,-4.8,0
3,1,-5,-4.7,0,-4.9,-4.7,0,-4.8,-4.7,0,...,0,4.7,-4.7,0,4.8,-4.7,0,4.9,-4.7,0
4,1,-5,-4.6,0,-4.9,-4.6,0,-4.8,-4.6,0,...,0,4.7,-4.6,0,4.8,-4.6,0,4.9,-4.6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999895,9999,-5,4.5,0,-4.9,4.5,0,-4.8,4.5,0,...,0,4.7,4.5,0,4.8,4.5,0,4.9,4.5,0
999896,9999,-5,4.6,0,-4.9,4.6,0,-4.8,4.6,0,...,0,4.7,4.6,0,4.8,4.6,0,4.9,4.6,0
999897,9999,-5,4.7,0,-4.9,4.7,0,-4.8,4.7,0,...,0,4.7,4.7,0,4.8,4.7,0,4.9,4.7,0
999898,9999,-5,4.8,0,-4.9,4.8,0,-4.8,4.8,0,...,0,4.7,4.8,0,4.8,4.8,0,4.9,4.8,0


In [3]:
# Load terrain T.O.W.R data
dataStep = pd.read_csv('../src/data/csv/NewDataset/02.Terrain_data_step.txt')
dataStep

,PathNum,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,300
0,1,-5,-5.0,0,-4.9,-5.0,0,-4.8,-5.0,0,...,0.0,4.7,-5.0,0.0,4.8,-5.0,0.0,4.9,-5.0,0.0
1,1,-5,-4.9,0,-4.9,-4.9,0,-4.8,-4.9,0,...,0.0,4.7,-4.9,0.0,4.8,-4.9,0.0,4.9,-4.9,0.0
2,1,-5,-4.8,0,-4.9,-4.8,0,-4.8,-4.8,0,...,0.0,4.7,-4.8,0.0,4.8,-4.8,0.0,4.9,-4.8,0.0
3,1,-5,-4.7,0,-4.9,-4.7,0,-4.8,-4.7,0,...,0.0,4.7,-4.7,0.0,4.8,-4.7,0.0,4.9,-4.7,0.0
4,1,-5,-4.6,0,-4.9,-4.6,0,-4.8,-4.6,0,...,0.0,4.7,-4.6,0.0,4.8,-4.6,0.0,4.9,-4.6,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999895,9999,-5,4.5,0,-4.9,4.5,0,-4.8,4.5,0,...,0.0,4.7,4.5,0.0,4.8,4.5,0.0,4.9,4.5,0.0
999896,9999,-5,4.6,0,-4.9,4.6,0,-4.8,4.6,0,...,0.0,4.7,4.6,0.0,4.8,4.6,0.0,4.9,4.6,0.0
999897,9999,-5,4.7,0,-4.9,4.7,0,-4.8,4.7,0,...,0.0,4.7,4.7,0.0,4.8,4.7,0.0,4.9,4.7,0.0
999898,9999,-5,4.8,0,-4.9,4.8,0,-4.8,4.8,0,...,0.0,4.7,4.8,0.0,4.8,4.8,0.0,4.9,4.8,0.0


In [4]:
# Load terrain T.O.W.R data
dataGap = pd.read_csv('../src/data/csv/NewDataset/02.Terrain_data_gap.txt')
dataGap

,PathNum,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,300
0,1,-5,-5.0,0,-4.9,-5.0,0,-4.8,-5.0,0,...,0,4.7,-5.0,0,4.8,-5.0,0,4.9,-5.0,0
1,1,-5,-4.9,0,-4.9,-4.9,0,-4.8,-4.9,0,...,0,4.7,-4.9,0,4.8,-4.9,0,4.9,-4.9,0
2,1,-5,-4.8,0,-4.9,-4.8,0,-4.8,-4.8,0,...,0,4.7,-4.8,0,4.8,-4.8,0,4.9,-4.8,0
3,1,-5,-4.7,0,-4.9,-4.7,0,-4.8,-4.7,0,...,0,4.7,-4.7,0,4.8,-4.7,0,4.9,-4.7,0
4,1,-5,-4.6,0,-4.9,-4.6,0,-4.8,-4.6,0,...,0,4.7,-4.6,0,4.8,-4.6,0,4.9,-4.6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999895,9999,-5,4.5,0,-4.9,4.5,0,-4.8,4.5,0,...,0,4.7,4.5,0,4.8,4.5,0,4.9,4.5,0
999896,9999,-5,4.6,0,-4.9,4.6,0,-4.8,4.6,0,...,0,4.7,4.6,0,4.8,4.6,0,4.9,4.6,0
999897,9999,-5,4.7,0,-4.9,4.7,0,-4.8,4.7,0,...,0,4.7,4.7,0,4.8,4.7,0,4.9,4.7,0
999898,9999,-5,4.8,0,-4.9,4.8,0,-4.8,4.8,0,...,0,4.7,4.8,0,4.8,4.8,0,4.9,4.8,0


In [5]:
# Load terrain T.O.W.R data
dataStair = pd.read_csv('../src/data/csv/NewDataset/02.Terrain_data_stair.txt')
dataStair

,PathNum,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,300
0,1,-5,-5.0,0,-4.9,-5.0,0,-4.8,-5.0,0,...,0,4.7,-5.0,0,4.8,-5.0,0,4.9,-5.0,0
1,1,-5,-4.9,0,-4.9,-4.9,0,-4.8,-4.9,0,...,0,4.7,-4.9,0,4.8,-4.9,0,4.9,-4.9,0
2,1,-5,-4.8,0,-4.9,-4.8,0,-4.8,-4.8,0,...,0,4.7,-4.8,0,4.8,-4.8,0,4.9,-4.8,0
3,1,-5,-4.7,0,-4.9,-4.7,0,-4.8,-4.7,0,...,0,4.7,-4.7,0,4.8,-4.7,0,4.9,-4.7,0
4,1,-5,-4.6,0,-4.9,-4.6,0,-4.8,-4.6,0,...,0,4.7,-4.6,0,4.8,-4.6,0,4.9,-4.6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999895,9999,-5,4.5,0,-4.9,4.5,0,-4.8,4.5,0,...,0,4.7,4.5,0,4.8,4.5,0,4.9,4.5,0
999896,9999,-5,4.6,0,-4.9,4.6,0,-4.8,4.6,0,...,0,4.7,4.6,0,4.8,4.6,0,4.9,4.6,0
999897,9999,-5,4.7,0,-4.9,4.7,0,-4.8,4.7,0,...,0,4.7,4.7,0,4.8,4.7,0,4.9,4.7,0
999898,9999,-5,4.8,0,-4.9,4.8,0,-4.8,4.8,0,...,0,4.7,4.8,0,4.8,4.8,0,4.9,4.8,0


In [ ]:
# Load terrain T.O.W.R data
dataSlope = pd.read_csv('../src/data/csv/NewDataset/02.Terrain_data_slope.txt')
dataSlope

In [ ]:
# Terrain_data[0] : X
# Terrain_data[1] : Y
# Terrain_data[2] : Z
# Each domain has 100x100 terrain information
# -> Xmin:-5.0, Xmax:5.0, Ymin:-5.0, Ymax:5.0, xy resolution 0.1
CommonXY = np.mgrid[-5:5:0.1, -5:5:0.1]
# Flat terrain
FlatTerrainXYZ  = dataFlat.values[:,1:].reshape(-1,100,100,3);  Terrain_data_Flat = np.transpose(FlatTerrainXYZ, (0,3,1,2))
Terrain_data_Flat[:,0]=CommonXY[1]; Terrain_data_Flat[:,1]=CommonXY[0]
# Step terrain
StepTerrainXYZ  = dataStep.values[:,1:].reshape(-1,100,100,3);  Terrain_data_Step = np.transpose(StepTerrainXYZ, (0,3,1,2))
Terrain_data_Step[:,0]=CommonXY[1]; Terrain_data_Step[:,1]=CommonXY[0]
# Gap terrain
GapTerrainXYZ   = dataGap.values[:,1:].reshape(-1,100,100,3);  Terrain_data_Gap  = np.transpose(GapTerrainXYZ, (0,3,1,2))
Terrain_data_Gap[:,0]=CommonXY[1];  Terrain_data_Gap[:,1]=CommonXY[0]

idx=randint(0,Terrain_data_Flat.shape[0])
print(idx)
plt.imshow(Terrain_data_Flat[idx,2])
plt.show()
idx=randint(0,Terrain_data_Step.shape[0])
print(idx)
plt.imshow(Terrain_data_Step[idx,2])
plt.show()
idx=randint(0,Terrain_data_Gap.shape[0])
print(idx)
plt.imshow(Terrain_data_Gap[idx,2])
plt.show()
# Save data
np.save('/content/drive/MyDrive/Colab Notebooks/data/Terrain_XYZ_flat',Terrain_data_Flat)
np.save('/content/drive/MyDrive/Colab Notebooks/data/Terrain_XYZ_step',Terrain_data_Step)
np.save('/content/drive/MyDrive/Colab Notebooks/data/Terrain_XYZ_gap',Terrain_data_Gap)

In [ ]:
# Load& Make merged Terrain data
Terrain_data_Flat = np.load('/content/drive/MyDrive/Colab Notebooks/data/Terrain_XYZ_flat.npy')
Terrain_data_Step = np.load('/content/drive/MyDrive/Colab Notebooks/data/Terrain_XYZ_step.npy')
Terrain_data_Gap = np.load('/content/drive/MyDrive/Colab Notebooks/data/Terrain_XYZ_gap.npy')

Terrain_data = np.concatenate((Terrain_data_Flat, Terrain_data_Step),axis=0)
del Terrain_data_Flat, Terrain_data_Step
Terrain_data = np.concatenate((Terrain_data, Terrain_data_Gap),axis=0)
del Terrain_data_Gap
# np.save('/content/drive/MyDrive/Colab Notebooks/data/Terrain_XYZ',Terrain_data)
np.save('/content/drive/MyDrive/Colab Notebooks/data/Terrain_XYZ_tensor',torch.tensor(Terrain_data, dtype=float))

In [ ]:
# Place Terrain data into dataloader
class GenTerrainDataset(Dataset):
 
  def __init__(self, TerrainData, transform=None):
      self.data = TerrainData.float()
      self.transform = transform
      
  def __getitem__(self, index):
      data = self.data
      if self.transform:
            data = self.transform(data[index])

      return index, data
 
  def __len__(self):
      return len(self.data)

transform = transforms.Compose(
    [
     transforms.ToPILImage(),
    #  transforms.Normalize(mean=[mean], std=[std]),
     transforms.ToTensor(),
    ])

# Load Terrain data
Terrain_data = np.load('/content/drive/MyDrive/Colab Notebooks/data/Terrain_XYZ.npy')
TESTterrain=torch.tensor(Terrain_data[:,2])

terrain_dataset = GenTerrainDataset(TESTterrain, transform=transform)
terrain_dataloader = DataLoader(terrain_dataset,batch_size=9,shuffle=True)

def show(img,nrow=3):
  # img = img*std + mean
  img_grid = torchvision.utils.make_grid(img,nrow=nrow).numpy()
  plt.figure()
  plt.imshow(np.transpose(img_grid, (1,2,0)))
  plt.axis('on')
  plt.show()

_, img = next(iter(terrain_dataloader))
show(img[:9])


In [ ]:
tf=transforms.ToPILImage()
tt=transforms.ToTensor()
plt.imshow(TESTterrain[-2].view(100,100))
plt.show()
plt.imshow(tt(tf(TESTterrain[-2])).view(100,100))
plt.show()

<p><img alt="Colaboratory logo" height="45px" src="/img/colab_favicon.ico" align="left" hspace="10px" vspace="0px"></p>

#<h1>Motion data organization</h1>

In [ ]:
# Load laikago T.O.W.R moition data
dataFlat = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/Hound/01.Motion_data_flat.txt')
dataFlat=dataFlat[:-23] # 마지막 값이 일부 누락됨
dataFlat

In [ ]:
# Load terrain T.O.W.R data
dataStep = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/Hound/01.Motion_data_step.txt')
dataStep

In [ ]:
# Load terrain T.O.W.R data
dataGap = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/Hound/01.Motion_data_gap.txt')
dataGap

In [ ]:
# Load terrain T.O.W.R data
dataGap = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/Hound/01.Motion_data_gap.txt')
dataGap

In [ ]:
# Load terrain T.O.W.R data
dataGap = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/Hound/01.Motion_data_gap.txt')
dataGap

In [ ]:
Data = [dataFlat, dataStep, dataGap]
PathData = []
for data in Data:
  Quad_data = torch.tensor(data.values)

  path=[0]*data.PathNum.max()
  path_idx=-1

  for idx in tqdm(range(len(Quad_data)), position=0, leave=True):
    if path_idx+1 == Quad_data[idx,0]-1:
      # Increase path index
      tmp=[]
      path_idx = path_idx+1
      # if path_idx == data.PathNum.max():
      #   # Skip last path
      #   break;
    # Revise negative z-forces
    domain=[-14, -11, -8, -5]
    for i in domain:
      if Quad_data[idx,i] < 0: 
        Quad_data[idx,i]=0.0

    # Revise z-forcese outliers
    MaxZforce=200
    for domain in range(-16,-4):
      if abs(Quad_data[idx,domain]) > MaxZforce: 
        Quad_data[idx,domain]=torch.sign(Quad_data[idx,domain])*MaxZforce

    if Quad_data[idx,0]==path_idx+1:
      # Rearrange data into (path, samples, variables)
      tmp.append(Quad_data[idx,2:].tolist())
      path[path_idx]=tmp

  print('\nOriginal data : ',np.shape(Quad_data))
  print(  'Path data     : ',np.shape(path))

  PathData.append(path)

np.save('/content/drive/MyDrive/Colab Notebooks/data/PathData',PathData)

In [ ]:
# Store path sizes
path = np.load('/content/drive/MyDrive/Colab Notebooks/data/PathData.npy', allow_pickle=True)
PathNUMs = [np.shape(path[idx])[0] for idx in range(np.shape(path)[0])]
MaxPathNum = sum(PathNUMs)
path_size=[]

print("\nMaxPathNum...")
for i in range(np.shape(path)[0]):
  for j in tqdm(range(np.shape(path[i])[0]), position=0, leave=True): 
    path_size.append(np.shape(path[i][j])[0])

 
MaxSampleNum = np.max(path_size)
Path_dim = np.shape(path[0][0][1])[0]
Paths = torch.zeros((MaxPathNum,MaxSampleNum,Path_dim))
 
# Convert path data into same size of data shape - padding with final states
print("\nConvert PathData...")
prevNUM=0
for i in range(np.shape(path)[0]):
  for j in tqdm(range(PathNUMs[i]), position=0, leave=True):
    for k in range(len(path[i][j])):
      Paths[j+prevNUM][k] = torch.FloatTensor(path[i][j][k])
    Paths[j+prevNUM][k:] = torch.FloatTensor(path[i][j][-1])
    prevNUM = sum(PathNUMs[:i])
 
# Path data shape
Paths_ori  = cp.deepcopy(Paths)
data_shape = (np.shape(Paths)[0], np.shape(Paths)[1], np.shape(Paths)[2])
print('\nPath data shape : ',data_shape)

# Save data
np.save('/content/drive/MyDrive/Colab Notebooks/data/Paths',Paths)

In [ ]:
plt.imshow(Paths.view(Paths.shape[0],-1))
plt.colorbar()
plt.show()

In [ ]:
idx=randint(0,MaxPathNum)
print(idx)
plt.title('Forces X')
plt.plot(Paths[idx,:,-16], label='LF')
plt.plot(Paths[idx,:,-13], label='RF')
plt.plot(Paths[idx,:,-10], label='LH')
plt.plot(Paths[idx,:,-7], label='RH')
plt.legend()
plt.show()

plt.title('Forces Y')
plt.plot(Paths[idx,:,-15], label='LF')
plt.plot(Paths[idx,:,-12], label='RF')
plt.plot(Paths[idx,:,-9], label='LH')
plt.plot(Paths[idx,:,-6], label='RH')
plt.legend()
plt.show()

plt.title('Forces Z')
plt.plot(Paths[idx,:,-14], label='LF')
plt.plot(Paths[idx,:,-11], label='RF')
plt.plot(Paths[idx,:,-8], label='LH')
plt.plot(Paths[idx,:,-5], label='RH')
plt.legend()
plt.show()


In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
force,=ax1.plot(Paths[idx,:,-14], label='Force')
contact,=ax2.plot(Paths[idx,:,-4], label='Contact', color='r')
graphs=[force, contact]
ax2.legend(graphs, [curve.get_label() for curve in graphs])
plt.title("LF")
plt.show()

fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
force,=ax1.plot(Paths[idx,:,-11], label='Force')
contact,=ax2.plot(Paths[idx,:,-3], label='Contact', color='r')
graphs=[force, contact]
ax2.legend(graphs, [curve.get_label() for curve in graphs])
plt.title("RF")
plt.show()

fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
force,=ax1.plot(Paths[idx,:,-8], label='Force')
contact,=ax2.plot(Paths[idx,:,-2], label='Contact', color='r')
graphs=[force, contact]
ax2.legend(graphs, [curve.get_label() for curve in graphs])
plt.title("LH")
plt.show()

fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
force,=ax1.plot(Paths[idx,:,-5], label='Force')
contact,=ax2.plot(Paths[idx,:,-1], label='Contact', color='r')
graphs=[force, contact]
ax2.legend(graphs, [curve.get_label() for curve in graphs])
plt.title("RH")
plt.show()